# Bench and MeasurementSession Integration Example

This notebook demonstrates the integration between:
- **Bench**: For instrument management and experiment context
- **MeasurementSession**: For parameter sweeps and measurement functions
- **Experiment**: For data capture
- **MeasurementDatabase**: For persistence

The example performs a transistor characterization using a bench configuration loaded from YAML, and executes a parameter sweep with the session.

In [ ]:
# Imports
import asyncio
import os
import numpy as np
from pathlib import Path

from pytestlab.bench import Bench
from pytestlab.measurements.session import MeasurementSession
import polars as pl

## Define the main experiment logic

This cell defines the main coroutine that will be run in the notebook.

In [ ]:
async def run_bench_session():
    # Path to the bench configuration YAML file
    bench_file = Path(__file__).parent / "session_bench.yaml" if '__file__' in globals() else Path().absolute() / "session_bench.yaml"
    print(f"Opening bench from: {bench_file}")
    
    # Open the bench - this initializes instruments and experiment context
    async with await Bench.open(bench_file) as bench:
        print(f"✅ Bench '{bench.name}' opened successfully")
        print(f"📋 Experiment: {bench.experiment.name}")
        
        # Verify database connection
        if bench.db:
            print(f"💾 Database connected: {bench.db.db_path}")
        
        # Create a measurement session that uses the bench
        async with MeasurementSession(bench=bench) as session:
            print(f"\n📊 Session created: {session.name}")
            
            # Define parameters for the sweep
            session.parameter("V_base", np.linspace(0.6, 1.0, 5), unit="V", notes="Base voltage")
            session.parameter("V_collector", np.linspace(0, 5, 10), unit="V", notes="Collector voltage")
            
            # Define a measurement function using bench instruments
            @session.acquire
            async def measure_transistor(V_base, V_collector, psu, dmm):
                """Measure transistor collector current at given base and collector voltages."""
                print(f"Setting V_base={V_base:.2f}V, V_collector={V_collector:.2f}V")
                
                # Set up base voltage on channel 1
                await psu.set_voltage(1, V_base)
                await psu.set_current(1, 0.05)  # 50mA limit for base
                
                # Set up collector voltage on channel 2
                await psu.set_voltage(2, V_collector)
                await psu.set_current(2, 0.5)   # 500mA limit for collector
                
                # Turn on outputs
                await psu.output(1, True)
                await psu.output(2, True)
                
                # Wait for circuit to stabilize
                await asyncio.sleep(0.1)
                
                # Measure collector current (in simulation mode this will return random values)
                result = await dmm.measure_current_dc()
                collector_current = result.values.nominal_value
                
                # Turn off outputs
                await psu.output(1, False)
                await psu.output(2, False)
                
                # Return measurements
                return {
                    "I_collector": collector_current,
                    "V_ce": V_collector,  # Collector-emitter voltage
                    "V_be": V_base,       # Base-emitter voltage
                }
            
            # Run the measurement sweep
            print("\n🔄 Starting measurement sweep...")
            experiment = await session.run(show_progress=True)
            print("\n✅ Measurement completed!")
            
            # Display results
            print("\n📈 Experiment data:")
            print(experiment.data.head(10))  # Show first 10 rows
            
            # Calculate transistor parameters
            print("\n🔍 Transistor characteristics:")
            df = experiment.data
            
            # Group by base voltage and show collector current range
            for v_base in sorted(set(df["V_base"].to_numpy().flatten())):
                base_rows = df.filter(pl.col("V_base") == v_base)
                i_collector_max = base_rows["I_collector"].max()
                print(f"Base voltage {v_base:.2f}V → Max collector current: {i_collector_max:.3f}A")
            
            # Report experiment database info
            print(f"\n💾 Experiment saved to database: {os.path.basename(bench.db.db_path)}")
            print(f"Experiment name: {bench.experiment.name}")
            print(f"Number of data points: {len(experiment.data)}")
            
            # Return experiment for further analysis if desired
            return experiment

## Run the experiment

This cell executes the main coroutine and stores the experiment result for further analysis.

In [ ]:
# Run the async experiment logic
experiment = asyncio.run(run_bench_session())

## Explore the experiment data

You can use the following cell to further analyze or visualize the experiment data interactively.

In [ ]:
# Display the first few rows of the experiment data
experiment.data.head(10)